In [1]:
import pandas as pd

from helpers import create_polling_id

pd.options.display.max_columns = None

# Argentina

Data extracted from Datos Gob

- https://datos.gob.ar/dataset/dine-resultados-provisionales-elecciones-2023
- https://datos.gob.ar/dataset/otros-resultados-provisorios-elecciones-generales-2019/archivo/otros_12.1

In [5]:
year = 2023
election = "deputy" # deputy

In [7]:
if year == 2023:
    df = pd.read_csv("../data/Argentina/2023/ResultadoElectorales_2023_Generales.csv")
    print(df["cargo_nombre"].unique())
    elections = {
        "first_round": "PRESIDENTE Y VICE",
        "runoff": "PRESIDENTE Y VICE",
        "deputy": "DIPUTADO NACIONAL"
    }
    df = df[df["cargo_nombre"] == elections[election]]
    df["polling_id"] = create_polling_id(df, ["distrito_id", "seccionprovincial_id", "seccion_id", "circuito_id", "mesa_id"])
    df = df.rename(columns={
        "votos_cantidad": "value",
        "agrupacion_nombre": "candidate"
    })
    if election in ["first_round", "runoff"]:
        df["candidate"] = df["candidate"].replace({
            "UNION POR LA PATRIA": "SERGIO MASSA", 
            "JUNTOS POR EL CAMBIO": "PATRICIA BULLRICH",
            "LA LIBERTAD AVANZA": "JAVIER MILEI",
            "FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD": "MYRIAM BREGMAN",
            "HACEMOS POR NUESTRO PAIS": "JUAN SCHIARETTI"
        })

elif year == 2019:
    df = pd.read_csv(f"../data/Argentina/{year}/mesas_agrp_politicas.dsv", delimiter="|")
    df_candidates = pd.read_csv("../data/Argentina/2019/descripcion_postulaciones.dsv", delimiter="|")
    df_candidates = df_candidates[df_candidates["NOMBRE_CATEGORIA"] == "Presidente y Vicepresidente de la República"]
    df_candidates = df_candidates[["CODIGO_CATEGORIA", "CODIGO_AGRUPACION", "NOMBRE_LISTA"]]

    df = pd.merge(df, df_candidates, on=["CODIGO_CATEGORIA", "CODIGO_AGRUPACION"])
    df = df.rename(columns={
        "VOTOS_AGRUPACION": "value",
        "NOMBRE_LISTA": "candidate",
        "CODIGO_MESA": "polling_id"
    })

df.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_43392\1002469568.py:2: DtypeWarning: Columns (9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/Argentina/2023/ResultadoElectorales_2023_Generales.csv")


['PRESIDENTE Y VICE' 'SENADOR NACIONAL' 'DIPUTADO NACIONAL'
 'GOBERNADOR Y VICE' 'SENADOR PROVINCIAL' 'DIPUTADO PROVINCIAL'
 'INTENDENTE' 'PARLAMENTO MERCOSUR NACIONAL'
 'PARLAMENTO MERCOSUR REGIONAL' 'CONCEJAL' 'JEFE/A DE GOBIERNO'
 'DIPUTADOS/AS DE LA CIUDAD AUTONOMA' 'MIEMBROS DE JUNTA COMUNAL']


,año,eleccion_tipo,eleccion_id,recuento_tipo,recuento_id,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,seccion_id,seccion_nombre,circuito_id,circuito_nombre,mesa_id,mesa_tipo,mesa_electores,cargo_id,cargo_nombre,agrupacion_id,candidate,lista_numero,lista_nombre,votos_tipo,value,polling_id
1483579,2023,GENERAL,2,PROVISORIO,1,NORMAL,1,Ciudad Autónoma de Buenos Aires,NaN,NaN,1,Comuna 01,00001,00001,1,NATIVOS,345,3,DIPUTADO NACIONAL,10503,UNION POR LA PATRIA,NaN,NaN,POSITIVO,89,1-###-1-00001-1
1483580,2023,GENERAL,2,PROVISORIO,1,NORMAL,1,Ciudad Autónoma de Buenos Aires,NaN,NaN,1,Comuna 01,00001,00001,1,NATIVOS,345,3,DIPUTADO NACIONAL,10504,JUNTOS POR EL CAMBIO,NaN,NaN,POSITIVO,65,1-###-1-00001-1
1483581,2023,GENERAL,2,PROVISORIO,1,NORMAL,1,Ciudad Autónoma de Buenos Aires,NaN,NaN,1,Comuna 01,00001,00001,1,NATIVOS,345,3,DIPUTADO NACIONAL,10501,LA LIBERTAD AVANZA,NaN,NaN,POSITIVO,44,1-###-1-00001-1
1483582,2023,GENERAL,2,PROVISORIO,1,NORMAL,1,Ciudad Autónoma de Buenos Aires,NaN,NaN,1,Comuna 01,00001,00001,1,NATIVOS,345,3,DIPUTADO NACIONAL,10502,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,NaN,NaN,POSITIVO,18,1-###-1-00001-1
1483583,2023,GENERAL,2,PROVISORIO,1,NORMAL,1,Ciudad Autónoma de Buenos Aires,NaN,NaN,1,Comuna 01,00001,00001,1,NATIVOS,345,3,DIPUTADO NACIONAL,0,NaN,NaN,NaN,EN BLANCO,14,1-###-1-00001-1


In [24]:
df.agrupacion_nombre.unique()

array(['UNION POR LA PATRIA', 'JUNTOS POR EL CAMBIO',
       'LA LIBERTAD AVANZA',
       'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD',
       'HACEMOS POR NUESTRO PAIS', nan], dtype=object)

In [26]:
data = df[df["votos_tipo"] == "POSITIVO"]

tt = data.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate", "value"]]

tt["rank"] = tt.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

In [30]:
tt.groupby("candidate").value.sum()

candidate
JAVIER MILEI         7884336
JUAN SCHIARETTI      1784315
MYRIAM BREGMAN        709932
PATRICIA BULLRICH    6267152
SERGIO MASSA         9645983
Name: value, dtype: int64

In [33]:
df_filtered = tt[["polling_id", "candidate", "value", "rank", "rate"]].copy()

df_filtered.to_csv(f"../data_output/Argentina/{year}_{election}.csv.gz", compression="gzip", index=False)

In [38]:
if year == 2023:
    df_location = df[["polling_id", "distrito_id", "distrito_nombre"]].drop_duplicates()
    df_location = df_location.rename(columns={"distrito_id": "district_id", "distrito_nombre": "district"})

elif year == 2019:
    df_location = df[["polling_id", "CODIGO_DISTRITO", "CODIGO_SECCION", "CODIGO_CIRCUITO"]].drop_duplicates()
    df_location = df_location.rename(columns={
        "CODIGO_DISTRITO": "district_id",
        "CODIGO_SECCION": "section_id",
        "CODIGO_CIRCUITO": "circuit_id"
    })

df_location.to_csv(f"../data_output/Argentina/{year}_{election}_location.csv.gz", compression="gzip", index=False)